In [35]:
from tensorflow.keras.models import load_model
from image_preprocessing import preprocess_multiclass
import numpy as np

loaded_multiclass_model = load_model('multiclass-model.h5')

### Prediction on a single image

In [171]:
import os
from PIL import Image
from sklearn.metrics import accuracy_score

image_path = '/Users/subhamgogoi/Pictures/testImages-multiClass/Bird/Bird_2.jpg'
img_name = os.path.basename(image_path)
image = Image.open(image_path)

image = image.resize((32,32))
image_array = np.array(image)
image_array = image_array / 255.0
image_array = np.reshape(image_array, (1,32,32,3))

prediction = loaded_multiclass_model.predict(image_array)
predicted_class = np.argmax(prediction)
class_names = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
class_index = {class_name:i for i, class_name in enumerate(class_names)}
print(f"Predicted class: {class_names[predicted_class]}")
# print(f'Class indices - {class_index}')

# Evaluate loss and accuracy
true_class_name = os.path.basename(os.path.dirname(image_path))
true_class_index = class_index[true_class_name]
true_class_index_hotencoded = tf.keras.utils.to_categorical(true_class_index, num_classes=10)
true_class_index_hotencoded = np.reshape(true_class_index_hotencoded, (1,10))

loss = tf.keras.losses.categorical_crossentropy(true_class_index_hotencoded, prediction)
print(f'Test loss on {img_name} - {loss[0]:.3f}')

accuracy = accuracy_score([true_class_index], [predicted_class])
print(f'Test accuracy on {img_name} - {accuracy}')

1/1 [==============================] - 0s 10ms/step
Predicted class: Bird
Test loss on Bird_2.jpg - 0.015
Test accuracy on Bird_2.jpg - 1.0


In [172]:
# print(prediction)

for i, prob in enumerate(prediction[0]):
    print(f"{class_names[i]} : {prob:.4f}")

Airplane : 0.0084
Car : 0.0001
Bird : 0.9849
Cat : 0.0022
Deer : 0.0003
Dog : 0.0038
Frog : 0.0002
Horse : 0.0001
Ship : 0.0000
Truck : 0.0000


### Prediction on a batch of images

In [36]:
# Preprocess the images
image_dir = '/Users/subhamgogoi/Pictures/testImages-multiClass'
image_batch, filenames = preprocess_multiclass(image_dir)

# Make predictions on the batch
predictions = loaded_multiclass_model.predict(image_batch)

# Define CIFAR-10 class names
class_names = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# Interpret and print predictions
predicted_classes = np.argmax(predictions, axis=1)
for pred_class, filename in zip(predicted_classes, filenames):
    print(f"{filename} is predicted as: {class_names[pred_class]}")

Populated 66 images


2024-09-02 12:41:52.751020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 1s 26ms/step
Cat_1.jpg is predicted as: Frog
Cat_2.jpg is predicted as: Dog
cat_3.jpg is predicted as: Cat
cat_7.jpg is predicted as: Deer
cat_6.jpg is predicted as: Frog
cat_4.jpg is predicted as: Cat
cat_5.jpg is predicted as: Cat
car_7.jpg is predicted as: Car
car_6.jpg is predicted as: Car
car_4.jpg is predicted as: Car
car_5.jpg is predicted as: Car
Car_1.jpg is predicted as: Truck
car_2.jpg is predicted as: Deer
car_3.jpg is predicted as: Car
dog_9.jpg is predicted as: Dog
dog_8.jpg is predicted as: Deer
dog_6.jpg is predicted as: Dog
dog_7.jpg is predicted as: Dog
dog_5.jpg is predicted as: Dog
dog_4.jpg is predicted as: Dog
dog_1.jpg is predicted as: Airplane
dog_3.jpg is predicted as: Deer
dog_2.jpg is predicted as: Dog
truck_6.jpg is predicted as: Truck
truck_5.jpg is predicted as: Truck
truck_4.jpg is predicted as: Airplane
truck_1.jpg is predicted as: Car
truck_3.jpg is predicted as: Truck
truck_2.jpg is predicted as: Truck
Bird_5.jpg 

### Evaluate loss and accuracy

In [37]:
from sklearn.metrics import accuracy_score
from image_labelling import auto_label_images
import tensorflow as tf

parent_dir = '/Users/subhamgogoi/Pictures/testImages-multiClass'
true_labels, filenames = auto_label_images(parent_dir)

true_labels_hot_encoded = tf.keras.utils.to_categorical(true_labels, num_classes=10)

In [38]:
# Check if the labels are correctly assigned to the images
# for filename, label in zip(filenames, true_labels):
#     print(f"{filename} : {label}")

In [39]:
import tensorflow as tf

loss = tf.keras.losses.categorical_crossentropy(true_labels_hot_encoded, predictions).numpy().mean()
print(f"Loss: {loss:.3f}")

test_acc = accuracy_score(true_labels, predicted_classes)
print(f"Test accuracy : {test_acc * 100 :.2f} %")

Loss: 8.005
Test accuracy : 9.09 %


In [40]:
import os
from collections import defaultdict

def count_images_per_class(image_dir):
    class_counts = defaultdict(int)

    # Traverse the directory and count images in each class subdirectory
    for class_name in os.listdir(image_dir):
        class_dir = os.path.join(image_dir, class_name)
        if os.path.isdir(class_dir):
            for filename in os.listdir(class_dir):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    class_counts[class_name] += 1

    return class_counts

class_counts = count_images_per_class('/Users/subhamgogoi/Pictures/testImages-multiClass')

for class_name, count in class_counts.items():
    print(f"{class_name}:{count}")

Cat:7
Car:7
Dog:9
Truck:6
Bird:6
Airplane:8
Ship:6
Frog:5
Horse:6
Deer:6
